# Graph Non-Isomorphism

In this chapter we construct a zero-knowledge protocol around graph non-isomorphism.

# What is a graph? What is isomorphism?

Check out the chapter on graph isomorphism.

# What is an interactive proof?

Peggy and Victor are engaged in an interactive proof.

There are two (random) graphs.

Peggy thinks that both graphs are structually different, ergo that both graphs are not isomorphic.  She wants to prove that to Victor.

Victor is sceptical and wants to see evidence. In particular he doesn't want to be convinced if Peggy is lying (both graphs are actually isomorphic).

Peggy wins if she convinces Victor. Victor wins by accepting only graphs that are actually different.

# Jupyter setup

Run the following snippet to set up your jupyter notebook for the workshop.

In [ ]:
import os
import sys

# Add project root so we can import local modules
root_dir = sys.path.append("..")
sys.path.append(root_dir)

# Graph Parameters

Choose the number of nodes and the number of edges for the first graph.

Feel free to experiment with different values.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# You can adjust the sliders any time

num_nodes = widgets.IntSlider(min=3, max=20, value=5, step=1, description="#Nodes")
num_edges = widgets.IntSlider(min=2, max=40, value=8, step=1, description="#Edges")

display(num_nodes)
display(num_edges)

# First Graph

Generate the first graph.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from local.graph import Mapping, random_graph, non_isomorphic_graph

# Rerun this cell to regenerate the first graph

graph1 = random_graph(num_nodes.value, num_edges.value)
print("First graph")
nx.draw(graph1, with_labels=True)
plt.show()

# Scenario

There are two scenarios in this interactive proof:

1. **Peggy is honest** and both graphs are structually different. She hopes to convince Victor of the true statement that both graphs are not isomorphic.
2. **Peggy is lying** and has no reason to believe that both graphs are different. There might even exist a translation between both graphs! Peggy still tries to fool Victor of the false statement that both graphs are not isomorphic.

Choose the good or the evil scenario. See how it affects the other cells further down.

In [ ]:
# You can adjust the selection any time
# Make sure to re-generate the second graph afterwards

is_isomorphic = widgets.Dropdown(
    options=[
        ("Second graph not isomorphic (honest Peggy 😇)", False),
        ("Second graph isomorphic (lying Peggy 😈)", True)],
    value=False,
    description="Scenario:",
)
is_isomorphic

# Second Graph

Now generate the second graph. In the good scenario it is structually different from the first graph. In the evil scenario, it is isomorphic.

In [ ]:
# Rerun this cell to generate a new second graph
# Rerun this cell after changing the scenario above

if is_isomorphic.value:
    from_1_to_2 = Mapping.shuffle_graph(graph1)
    graph2 = from_1_to_2.apply_graph(graph1)
    print("Second graph [isomorphic to first graph]")
    print("Translation: {}".format(from_1_to_2))
else:
    graph2 = non_isomorphic_graph(graph1)
    print("Second graph [not isomorphic to first graph]")
    
nx.draw(graph2, with_labels=True)
plt.show()

# Protocol

The proof is simple:

Victor randomly chooses between graph one or two, shuffles it and sends the resulting shuffled graph to Peggy.

Peggy has to decide if graph one or two was shuffled. If graph one and two are structually different, then the shuffled graph is always isomorphic to either one but never both. So Peggy checks for isomorphism and answers.

Victor checks if Peggy answered correctly.

In [ ]:
import random

class Peggy:
    def __init__(self, graph1: nx.Graph, graph2: nx.Graph):
        self.graph1 = graph1
        self.graph2 = graph2
    
    def distinguish(self, shuffled_graph: nx.Graph) -> nx.Graph:
        if nx.is_isomorphic(self.graph1, shuffled_graph):
            return 0
        else:
            assert nx.is_isomorphic(self.graph2, shuffled_graph)
            return 1

class Victor:
    def __init__(self, graph1: nx.Graph, graph2: nx.Graph):
        self.graphs = [graph1, graph2]
    
    def shuffled_graph(self) -> nx.Graph:
        self.chosen_index = random.randrange(0, 2)
        chosen_graph = self.graphs[self.chosen_index]
        shuffle = Mapping.shuffle_graph(chosen_graph)
        shuffled_graph = shuffle.apply_graph(chosen_graph)
        
        return shuffled_graph
    
    def verify(self, index: int) -> bool:
        return index == self.chosen_index

# Single Exchange

Let's run a couple of exchanges to get familiar. Feel free to experiment with the graph parameters and the scenario. The system is random so there could be a different outcome next time you run with the same settings 😉

In [ ]:
# Feel free to run this multiple times

peggy = Peggy(graph1, graph2)
victor = Victor(graph1, graph2)

shuffled_graph = victor.shuffled_graph()
index = peggy.distinguish(shuffled_graph)

# Victor is convinced
if victor.verify(index):
    # Two graphs were isomorphic (evil)
    if is_isomorphic.value:
        print("Convinced 👌 (Victor was fooled)")
    # Two graphs were non-isomorphic (good)
    else:
        print("Convinced 👌 (expected)")
# Victor is not convinced
else:
    # Two graphs were isomorphic (evil)
    if is_isomorphic.value:
        print("Not convinced... 🤨 (expected)")
    # Two graphs were non-isomorphic (good)
    else:
        print("Not convinced... 🤨 (Peggy was dumb)")

# Completeness

Does Peggy have a chance to convince Victor if she is honest? Let's run the protocol a couple of times and see how often she manages to convince him.

In [ ]:
num_exchanges_complete = widgets.IntSlider(min=10, max=1000, value=10, step=10, description="#Exchanges")
num_exchanges_complete

In [ ]:
graph3 = non_isomorphic_graph(graph1)

honest_peggy = Peggy(graph1, graph3)
victor = Victor(graph1, graph3)

peggy_success = 0

for _ in range(num_exchanges_complete.value):
    shuffled_graph = victor.shuffled_graph()
    index = honest_peggy.distinguish(shuffled_graph)

    if victor.verify(index):
        peggy_success += 1
        
peggy_success_rate = peggy_success / num_exchanges_complete.value * 100

print("Running {} exchanges".format(num_exchanges_complete.value))
print("Honest Peggy wins {0:0.2f}% of the time".format(peggy_success_rate))
print()

assert peggy_success_rate == 100
print("Peggy always wins if she is honest")

# Soundness

Now, does Victor have a chance to expose Peggy if she is lying? Actually, Peggy can pass his test 50% of the time by random guessing!

There is an easy way to boost Victor's confidence: run the protocol for **multiple rounds**. Victor shuffles one of the two graphs, Peggy has to say which graph was shuffled; repeat. Each round the shuffling and the chosen graph will be different. Peggy has to meet Victor's challenge **every time** while he has to expose her cheating **only once**. He has a much easier time and thus gains in confidence.

The chance that Victor gets fooled after `n` rounds is `0.5^i`, which decreases exponentially in `i`.

In [ ]:
num_exchanges_sound = widgets.IntSlider(min=10, max=1000, value=10, step=10, description="#Exchanges")
num_rounds = widgets.IntSlider(min=1, max=10, value=1, step=1, description="#Rounds")

display(num_exchanges_sound)
display(num_rounds)

In [ ]:
# Translation between graphs 1 and 4
# Both graphs are isomorphic!
from_1_to_4 = Mapping.shuffle_graph(graph1)
graph4 = from_1_to_4.apply_graph(graph1)

lying_peggy = Peggy(graph1, graph4)
victor = Victor(graph1, graph4)

victor_success = 0

for _ in range(num_exchanges_sound.value):
    for _ in range(num_rounds.value):
        shuffled_graph = victor.shuffled_graph()
        index = lying_peggy.distinguish(shuffled_graph)
    
        if not victor.verify(index):
            victor_success += 1
            break
            
victor_success_rate = victor_success / num_exchanges_sound.value * 100

print("Running {} exchanges with {} rounds each".format(num_exchanges_sound.value, num_rounds.value))
print("Victor wins against lying Peggy {0:0.2f}% of the time".format(victor_success_rate))
print()

if victor_success_rate < 50:
    print("Victor loses quite often for a small number of rounds")
elif victor_success_rate < 90:
    print("Victor gains more confidence with each added round")
else:
    print("At some point it is basically impossible to fool Victor")

# Zero-Knowledge

There is no secret involved in this proof, but does Victor learn anything from his exchange with Peggy (besides the fact that both graphs are structually different)?

If we look at the transcript, it is a shuffled graph and the index of the graph that was shuffled. Anybody can replicate that! If we choose a random index and shuffle that graph, the outcome should be the same.

Let's do a chi-square test to see if the real and fake transcripts follow the same distribution.

In [ ]:
num_transcripts = widgets.IntSlider(min=1000, max=50000, value=10000, step=1000, description="#Transcripts")
num_transcripts

In [ ]:
from typing import Tuple
import local.stats as stats

# Make sure to run this for small graphs (adjust the graph parameters)
# Large graphs can lead to errors
# The list of critical chi-square values might be too short for the large number of degrees of freedom
# Large graphs also require very many transcripts, which is slow

peggy = Peggy(graph1, graph2)
victor = Victor(graph1, graph2)

def real_transcript() -> Tuple:
    shuffled_graph = victor.shuffled_graph()
    index = peggy.distinguish(shuffled_graph)
    
    return tuple(shuffled_graph.edges()), index


def fake_transcript() -> Tuple:
    index = random.randrange(0, 2)
    chosen_graph = [graph1, graph2][index]
    shuffle = Mapping.shuffle_graph(chosen_graph)
    shuffled_graph = shuffle.apply_graph(chosen_graph)
    
    return tuple(shuffled_graph.edges()), index


real_samples = [real_transcript() for _ in range(num_transcripts.value)]
fake_samples = [fake_transcript() for _ in range(num_transcripts.value)]

# The chi-square test is only valid if most bins are filled
# Increase the number of transcripts if there are too many empty bins

null_hypothesis = stats.chi_square_equal(real_samples, fake_samples)
print()

if null_hypothesis:
    print("Real and fake transcripts are the same distribution.")
    print("Victor learns nothing 👌")
else:
    print("Real and fake transcripts are different distributions.")
    print("Victor might learn something 😧")

stats.plot_comparison(real_samples, fake_samples, "real", "fake")